## 6.01 - Supervised Learning Model Comparison

Recall the "data science process."

1. Define the problem.
2. Gather the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.
Most of the questions requiring a written response can be written in 2-3 sentences.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. When predicting `e401k`, you may use the entire dataframe if you wish.

In [41]:
#imports 
import pandas as pd
import numpy as np

from math import sqrt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR, SVC

import warnings
warnings.simplefilter(action='ignore')

### Step 2: Gather the data.

##### 1. Read in the data from the repository.

In [4]:
df = pd.read_csv('401ksubs.csv')

df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


##### 2. What are 2-3 other variables that, if available, would be helpful to have?

- It would be helpful to have variables that predict financial stability, such as a) how much money is in one's savings account, b) how much money is in one's checking account, and c) what is one's credit score.
- Owning a house versus renting a residence may be a good indicator of someone able and willing to save money. (The same goes for a car, too!)
- Knowing whether or not the person has made investments into stocks, bonds, or Certificates of Deposits (CDs) would be helpful to predict eligibility for 401(k)s.


##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

 It's unethical to use race to determine who should qualify for any particiular product or service. Someone should not be disqualified from a model on the basis of race as someone's race does not determine their eligibility for 401k's and related finicial qualifications. 

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

Answer: When predicting income I will obviously not be needing or using the income or income squared columns. 

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs might have done this!

Answer: Age and income have both been squared i.e., feature engineered. Assuming that this was done by subject-matter experts, it's likely that they found there's some quadratic (squared) relationship between income and eligibility for a 401k. (The same goes for age and eligibility for a 401k.) Perhaps people who are older or have higher incomes are exponentially more likely to be eligible for an IRA, so these terms account for that nonlinearity.

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

Answer: The income column is described as income squared when that does not appear to be the case, similarly  the age column is also described as age squared when it just is age. There is also a seperate income squared and seperate age squared column.

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

Answer:

- a multiple linear regression model (Yes, we can understand influence of features.)
- a $k$-nearest neighbors model (No, we cannot understand influence of features.)
- a decision tree (Yes, we can understand influence of features.)
- a set of bagged decision trees (Yes, we can understand influence of features.)
- a random forest (Yes, we can understand influence of features.)
- a set of extremely randomized trees (Yes, we can understand influence of features.)
- an Adaboost model (Yes, we can understand influence of features.)
- an XGBoost model (Yes, we can understand influence of features.)
- a support vector regressor (Yes, we can understand influence of features.)

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector regressor
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [6]:
#Scale our dataframe 

scaled_df = pd.DataFrame(StandardScaler().fit_transform(df),
                        columns = df.columns)

scaled_df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,-0.803173,-1.082858,-1.300887,-0.506898,-0.104886,-1.235500,-0.226651,-0.617776,1.712236,-0.648965,-0.216227
1,1.245062,0.912268,-1.300887,1.972784,-0.590372,-1.235500,2.109561,1.618709,-0.584032,0.542404,-0.634940
2,-0.803173,-1.095810,0.768706,-0.506898,0.283503,-0.580086,-0.298179,-0.617776,-0.584032,-0.651671,0.158941
3,-0.803173,2.475242,0.768706,1.972784,0.283503,-0.580086,0.042656,-0.617776,-0.584032,2.550909,0.158941
4,-0.803173,-0.690807,-1.300887,-0.506898,1.157377,-1.235500,-0.009720,-0.617776,-0.584032,-0.536366,1.133706


In [14]:
X = scaled_df.drop(columns = ['e401k', 'p401k', 'pira', 'inc', 'incsq'])

y = scaled_df['inc']

In [7]:
#Using all features besides inc, inc squared and 401k vars.

X_train, X_test, y_train, y_test = train_test_split(scaled_df.drop(columns = ['e401k', 'p401k', 'pira', 'inc', 'incsq']),
                                                    scaled_df['inc'],
                                                    test_size = .2,
                                                    random_state = 42)

## Models

In [10]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train, y_train)

cart_reg = DecisionTreeRegressor()
cart_reg.fit(X_train, y_train)

bagged_reg = BaggingRegressor()
bagged_reg.fit(X_train, y_train)

random_forest_reg = RandomForestRegressor()
random_forest_reg.fit(X_train, y_train)

adaboost_reg = AdaBoostRegressor()
adaboost_reg.fit(X_train, y_train)

support_vector_reg = SVR()
support_vector_reg.fit(X_train, y_train)

SVR()

##### 9. What is bootstrapping?

Answer: Bootstrapping is random resampling with reaplcement. You take an original data sample, take many sub-samples with replacement, and those are bootstrapped samples.  We usually use it in order to simulate many different samples or to empirically estimate the sampling distribution of a statistic.

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

Answer: With a set of bagged decision trees, we have bootstrapped $k$ different samples and grown one decision tree on each bootstrapped sample, then our predictions are aggregated. With one decision tree, we only use the original sample and grow exactly one decision tree. No aggregation of predictions occur.

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

Answer: In bagged decision trees, every variable is considered as a "candidate" for splitting at each node in the decision tree. In random forests, a random subset of variables is considered as candidataes for splitting at each node. 

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

Answer: In a random forest, we randomly select which features go into each split. This effectively makes our individual decision trees less correlated, which decreases the variance of our predictions after aggregating the different decision trees. Thus, a random forest usually has less variance than bagged decision trees.



## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

In [29]:
def rmse_score(model, X_train, X_test, y_train, y_test):
    mse_train = mean_squared_error(y_true = y_train,
                                  y_pred = model.predict(X_train))
    mse_test = mean_squared_error(y_true = y_test,
                                  y_pred = model.predict(X_test))
    rmse_train = mse_train ** 0.5
    rmse_test = mse_test ** 0.5
    
    print("The training RMSE for " + str(model) + " is: " + str(rmse_train))
    print("The testing RMSE for " + str(model) + " is: " + str(rmse_test))
    return (rmse_train, rmse_test)

In [30]:
#Linear Regression
rmse_score(linear_reg, X_train, X_test, y_train, y_test)

The training RMSE for LinearRegression() is: 0.8370830332453489
The testing RMSE for LinearRegression() is: 0.8675193605893167


(0.8370830332453489, 0.8675193605893167)

In [31]:
#KNN 
rmse_score(knn_reg, X_train, X_test, y_train, y_test)

The training RMSE for KNeighborsRegressor() is: 0.6860905782308746
The testing RMSE for KNeighborsRegressor() is: 0.8373266164049329


(0.6860905782308746, 0.8373266164049329)

In [32]:
#Decision Tree 


rmse_score(cart_reg, X_train, X_test, y_train, y_test)

The training RMSE for DecisionTreeRegressor() is: 0.09397820060704348
The testing RMSE for DecisionTreeRegressor() is: 1.1247456481932119


(0.09397820060704348, 1.1247456481932119)

In [33]:
#Bagging Regressor

rmse_score(bagged_reg, X_train, X_test, y_train, y_test)

The training RMSE for BaggingRegressor() is: 0.3633811622127923
The testing RMSE for BaggingRegressor() is: 0.874687992042935


(0.3633811622127923, 0.874687992042935)

In [34]:
#Random Forests

rmse_score(random_forest_reg, X_train, X_test, y_train, y_test)

The training RMSE for RandomForestRegressor() is: 0.32040091355927897
The testing RMSE for RandomForestRegressor() is: 0.8427712208089381


(0.32040091355927897, 0.8427712208089381)

In [35]:
#AdaBoost 

rmse_score(adaboost_reg, X_train, X_test, y_train, y_test)

The training RMSE for AdaBoostRegressor() is: 0.9206854207488931
The testing RMSE for AdaBoostRegressor() is: 0.960467791770503


(0.9206854207488931, 0.960467791770503)

In [36]:
#SVR 
rmse_score(support_vector_reg, X_train, X_test, y_train, y_test)

The training RMSE for SVR() is: 0.7858885593080802
The testing RMSE for SVR() is: 0.8206525603025999


(0.7858885593080802, 0.8206525603025999)

Putting the training and testing RMSE in a table so we can directly compare them all:

|           Model          | Training RMSE | Testing RMSE |
|:------------------------:|:-------------:|:------------:|
|     Linear Regression    |     0.837     |     0.868    |
|    k-Nearest Neighbors   |     0.686     |     0.837    |
|       Decision Tree      |     0.093     |     1.124    |
|   Bagged Decision Trees  |     0.363     |     0.874    |
|       Random Forest      |     0.320     |     0.842    |
|         AdaBoost         |     0.902     |     0.960    |
| Support Vector Regressor |     0.786     |     0.820    |

##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?


Answer: It appears that every model is overfit to the training data. We recognize this because our testing RMSE is worse (higher) than our training RMSE. Our models are generalizing poorly to held-out/unseen data.

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

I would pick AdaBoost or Linear Regression on the basis that the scores are more consistent across train and test when compared to other models. LinearRegression is similer to understand yet AdaBoost yeilds higher scores.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

- Tune Models hyperparamters using GridSearch 
- Further feature engineering of the data 
- Transform y variable further. 

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

Answer: We're predicting whether or not someone is eligible for a 401k. If we include whether or not someone already possesses a 401k, then every person with a 401k must by definition be eligible for a 401k. However, this probably doesn't contribute to our understanding of what makes someone eligible for a 401k and will only confound what we're actually interested in studying.

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

- a logistic regression model (Yes, we can predict whether or not one is eligible for a 401(k).)
- a $k$-nearest neighbors model (Yes, we can predict whether or not one is eligible for a 401(k).)
- a Naive Bayes model (Yes, we can predict whether or not one is eligible for a 401(k).)
- a decision tree (Yes, we can predict whether or not one is eligible for a 401(k).)
- a set of bagged decision trees (Yes, we can predict whether or not one is eligible for a 401(k).)
- a random forest (Yes, we can predict whether or not one is eligible for a 401(k).)
- a set of extremely randomized trees (Yes, we can predict whether or not one is eligible for a 401(k).)
- an Adaboost model (Yes, we can predict whether or not one is eligible for a 401(k).)
- an XGBoost model (Yes, we can predict whether or not one is eligible for a 401(k).)
- a support vector classifier (Yes, we can predict whether or not one is eligible for a 401(k).)

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector classifier
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

df.head()

In [38]:
#train test split 

X_train, X_test, y_train, y_test = train_test_split(scaled_df.drop(columns = ['e401k', 'p401k']),
                                                    [1 if scaled_df['e401k'][i] > 0 else 0 for i in range(scaled_df.shape[0])],
                                                    ## I ran the above line because when I scaled e401k, e401k was no longer binary!
                                                    ## I needed to turn my Y vector into a discrete variable.
                                                    test_size = .2,
                                                    random_state = 42)

In [39]:
np.random.seed(42)

In [42]:
#models 

logreg_class = LogisticRegression()
logreg_class.fit(X_train, y_train)

knn_class = KNeighborsClassifier()
knn_class.fit(X_train, y_train)

cart_class = DecisionTreeClassifier()
cart_class.fit(X_train, y_train)

bagged_class = BaggingClassifier()
bagged_class.fit(X_train, y_train)

random_forest_class = RandomForestClassifier()
random_forest_class.fit(X_train, y_train)

adaboost_class = AdaBoostClassifier()
adaboost_class.fit(X_train, y_train)

support_vector_class = SVC()
support_vector_class.fit(X_train, y_train)


SVC()

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

In [179]:
from sklearn.metrics import confusion_matrix

In [186]:
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()

print(f"True Negative (True Non - Elibility for 401K): {tn}")
print(f"True Positive (True Eligibility for 401K ): {tp}")
print(f"False Negative (Model says no eligile for 401k, but is actually eligible ): {fn}")
print(f"False Positive (Model says is eligiile for 401 k but is not): {fp}")

True Negative (True Non - Elibility for 401K): 1272
True Positive (True Eligibility for 401K ): 259
False Negative (Model says no eligile for 401k, but is actually eligible ): 638
False Positive (Model says is eligiile for 401 k but is not): 150


##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

We would rather mnimize false positives because we want as many people who are eligible for 401k's to recieve advertising for 401k's. Here we are wanting to minimize false negatives.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

Since we want to optimize for false negatives, we should optimize sensitvity.

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

The f1 score conveys the balance between the precision and the recall. This will measure our models accuracy on two different metrics which should help it balance out. 

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

In [43]:
def f1_scorer(model, X_train, X_test, y_train, y_test):
    f1_train = f1_score(y_true = y_train,
                        y_pred = model.predict(X_train))
    f1_test = f1_score(y_true = y_test,
                       y_pred = model.predict(X_test))
    
    print("The training F1-score for " + str(model) + " is: " + str(f1_train))
    print("The testing F1-score for " + str(model) + " is: " + str(f1_test))
    return (f1_train, f1_test)

In [44]:
print(f1_scorer(logreg_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(knn_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(cart_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(bagged_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(random_forest_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(adaboost_class, X_train, X_test, y_train, y_test))
print()
print(f1_scorer(support_vector_class, X_train, X_test, y_train, y_test))

The training F1-score for LogisticRegression() is: 0.4727870199219552
The testing F1-score for LogisticRegression() is: 0.4777870913663035
(0.4727870199219552, 0.4777870913663035)

The training F1-score for KNeighborsClassifier() is: 0.653122648607976
The testing F1-score for KNeighborsClassifier() is: 0.4977511244377811
(0.653122648607976, 0.4977511244377811)

The training F1-score for DecisionTreeClassifier() is: 1.0
The testing F1-score for DecisionTreeClassifier() is: 0.4654696132596685
(1.0, 0.4654696132596685)

The training F1-score for BaggingClassifier() is: 0.9678891033514652
The testing F1-score for BaggingClassifier() is: 0.49413604378420645
(0.9678891033514652, 0.49413604378420645)

The training F1-score for RandomForestClassifier() is: 1.0
The testing F1-score for RandomForestClassifier() is: 0.5329295987887964
(1.0, 0.5329295987887964)

The training F1-score for AdaBoostClassifier() is: 0.5621436716077537
The testing F1-score for AdaBoostClassifier() is: 0.568848758465011

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

Answer: We want our $F_1$ score to be as high as possible. Thus, overfitting occurs when we have a high training $F_1$ score and a low testing $F_1$ score. Models that appear to be overfit are the $k$-nearest neighbors classifier, the decision tree, the bagged decision trees, the random forest, and (slightly) the support vector classifier.

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

I would choose Adaboost. Adaboost has shown consistent scores accross train and test data and has relatively high R-sqaured values

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

- Hyperparameter tuning 
- Feature Engineering
- Using other libraries such a polynomial features. 

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

In [207]:
#regression

In [208]:
features = ['marr', 'male', 'age', 'fsize', 'nettfa']

y = df['inc']

X = df[features]

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [211]:
import statsmodels.api as sm

In [212]:
X = sm.add_constant(X)

X.head()

,const,marr,male,age,fsize,nettfa
0,1.0,0,0,40,1,4.575
1,1.0,0,1,35,1,154.000
2,1.0,1,0,44,2,0.000
3,1.0,1,1,44,2,21.800
4,1.0,0,0,53,1,18.450


In [213]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    inc   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     663.6
Date:                Mon, 01 Feb 2021   Prob (F-statistic):               0.00
Time:                        10:45:06   Log-Likelihood:                -41252.
No. Observations:                9275   AIC:                         8.252e+04
Df Residuals:                    9269   BIC:                         8.256e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.8851      1.060     24.422      0.000      23.807      27.963
marr          20.3100      0.558     36.422      0.000      19.217      21.403
male           3.4407      0.582      5.907      0.000       2.299       4.582
age            0.0380      0.022      1.766      0.077      -0.004       0.080
fsize         -1.4244      0.174     -8.200      0.000      -1.765      -1.084
nettfa         0.1284      0.003     37.250      0.000       0.122       0.135
==============================================================================
Omnibus:                     2341.739   Durbin-Watson:                   1.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10549.893
Skew:                           1.163   Prob(JB):                         0.00
Kurtosis:                       7.679   Cond. No.                         350.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Answer: Based on my summary statistics and their respective p-values, it appears that NETTFA likely is the largest predicting variable of income.